In [51]:
%autosave 60

Autosaving every 60 seconds


In [52]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.metrics import accuracy_score, roc_auc_score,recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# стороняя библиотека для работы с несбалансированными датасетами
# pip install imblearn
from imblearn.over_sampling import SMOTE

# настройки отображения графиков
%config InlineBackend.figure_format = 'svg' 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')
%matplotlib inline

# увеличим  размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

# для воспроизводимости
r_state = 11

In [53]:
data_path = './data/creditcard.csv'

In [54]:
df = pd.read_csv(data_path)

https://beckernick.github.io/oversampling-modeling/

In [55]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [56]:
# sns.distplot(df[df['Class']==1]['Time'],200);

In [57]:
# sns.countplot(df['Class']);

In [58]:
# plt.plot(X_old[['V1','V2']].values[Y_old == 0,0], X_old[['V1','V2']].values[Y_old == 0, 1], 'bo', label='non_fraud')
# plt.plot(X_old[['V1','V2']].values[Y_old == 1,0], X_old[['V1','V2']].values[Y_old == 1, 1], 'go', label='fraud')
# plt.legend(loc=0);

In [111]:
def all_metrics_validation(model,X,Y):
    print("Accuracy score: "+ str(accuracy_score(Y,model.predict(X))))
    print("ROC AUC score: "+ str(roc_auc_score(Y,model.predict_proba(X)[:,1]))) # нас интересуют положительные результаты
    print("Recall score: "+ str(recall_score(Y,model.predict(X))))

In [148]:
def create_new_data(data,smote=False):

    fraud = data[(data['Class']==1)]
    not_fraud = data[(data['Class']==0)]
    new_df=pd.concat([fraud,data[:len(fraud)]])      
    X = new_df.drop(['Class','Time'], axis=1)
    Y = new_df['Class']
    return X,Y

In [129]:
# %%time
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# tsne = TSNE(random_state=17)
# tsne_representation = tsne.fit_transform(X_scaled)
# plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1]);

In [130]:
df = df.sample(n=100000,random_state = r_state)

In [131]:
def smote(x,y):
    sm = SMOTE(random_state=12, ratio =  'minority')
    X,Y = sm.fit_sample(x, y)
    Y=pd.Series(Y)
    return X,Y

In [132]:
X = df.drop(['Time','Class'],axis=1)
Y = df['Class']

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [134]:
X_train,y_train = smote(X_train,y_train)
X_test,y_test = smote(X_test,y_test)

In [135]:
y_train.value_counts()

1    69876
0    69876
dtype: int64

# Случайный лес

In [99]:
%%time
parameters = {'max_features': ['auto'], 'min_samples_leaf': [1, 3],'max_depth': [1,2,3,10],'n_estimators':[10],'n_jobs':[-1]}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

rfc = RandomForestClassifier()

gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1)

gcv.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    4.3s finished


CPU times: user 8.1 s, sys: 135 ms, total: 8.24 s
Wall time: 6.1 s


In [100]:
model = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

({'max_depth': 10,
  'max_features': 'auto',
  'min_samples_leaf': 1,
  'n_estimators': 10,
  'n_jobs': -1},
 0.9955560962344893)

array([[9.99081815e-01, 9.18185468e-04],
       [9.99580335e-01, 4.19665328e-04],
       [9.99428375e-01, 5.71624974e-04],
       ...,
       [9.98112925e-01, 1.88707511e-03],
       [9.92471871e-01, 7.52812877e-03],
       [9.90526440e-01, 9.47355964e-03]])

# Посмотри про индексацию двумерных массивов

In [113]:
all_metrics_validation(model,X_test,y_test)

Accuracy score: 0.9246776671788363
ROC AUC score: 0.974976419969922
Recall score: 0.850624624223395


# Метод ближайших соседей

In [114]:
%%time
parameters = {'n_neighbors': range(9,10),'n_jobs':[-1]}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

knn = KNeighborsClassifier()

gcv = GridSearchCV(knn, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   13.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   13.9s finished


CPU times: user 479 ms, sys: 83.9 ms, total: 563 ms
Wall time: 14.4 s


In [115]:
model = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

({'n_jobs': -1, 'n_neighbors': 9}, 0.9990983383664181)

In [116]:
all_metrics_validation(model, X,Y)

Accuracy score: 0.99456
ROC AUC score: 0.980948087286508
Recall score: 0.9424083769633508


# Логистическая регрессия

In [117]:
# poly = PolynomialFeatures(degree=7)
# X_poly = poly.fit_transform(X)

In [164]:
%%time
parameters = {'C': np.linspace(25,30,20),'n_jobs':[-1]}

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=r_state)

logit = LogisticRegression()
gcv = GridSearchCV(logit, parameters, n_jobs=-1, cv=skf, verbose=1,scoring='recall')

gcv.fit(X_train, y_train)





Fitting 2 folds for each of 20 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   25.3s finished


CPU times: user 3.67 s, sys: 112 ms, total: 3.79 s
Wall time: 27.1 s


In [165]:
model = gcv.best_estimator_
gcv.best_params_ , gcv.best_score_

({'C': 29.473684210526315, 'n_jobs': -1}, 0.9253677943786135)

In [166]:
all_metrics_validation(model, X,Y)

Accuracy score: 0.9698
ROC AUC score: 0.9808624787480213
Recall score: 0.9057591623036649


In [167]:
X_t,Y_t = create_new_data(df)

In [168]:
X_t.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,...,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000,382.000000
mean,-2.153589,1.765861,-3.380186,2.295885,-1.268401,-0.759194,-2.670236,0.115026,-1.253716,-2.745200,...,0.168617,0.349003,-0.018510,-0.055983,-0.088412,0.036818,0.024731,0.101648,0.044951,85.292592
std,5.290148,3.595033,6.169350,3.231782,3.959798,1.690095,5.773232,4.798290,2.337736,4.628239,...,1.049102,2.987917,1.189524,1.275251,0.575813,0.669748,0.503479,1.067081,0.463985,183.873917
min,-28.709229,-8.402154,-30.177317,-4.521976,-20.346228,-6.406267,-43.557242,-41.044261,-13.320155,-24.588262,...,-4.128186,-22.797604,-8.887017,-15.981649,-2.422940,-3.536716,-1.152671,-7.263482,-1.936276,0.000000
25%,-2.419046,-0.120912,-5.217856,-0.093564,-1.468953,-1.723731,-3.017517,-0.201591,-2.056335,-4.690556,...,-0.186656,-0.187974,-0.550510,-0.242952,-0.439746,-0.296024,-0.306575,-0.038134,-0.044519,1.000000
50%,-0.702877,0.949245,-1.367779,1.368524,-0.322770,-0.671313,-0.784901,0.147658,-0.735379,-0.862046,...,0.048592,0.134125,-0.007084,-0.047422,-0.014790,0.101266,-0.047764,0.058007,0.035824,12.360000
75%,1.082233,2.753808,0.405000,4.284724,0.509672,0.051076,0.298841,0.761387,0.069632,0.165280,...,0.401989,0.563740,0.538718,0.178962,0.332452,0.418470,0.349890,0.475956,0.237222,95.705000
max,2.341610,22.057729,3.453140,11.927512,11.095089,6.474115,5.802537,18.329406,9.099082,13.249433,...,11.059004,27.202839,8.316275,5.466230,1.202293,1.999499,2.745261,2.352333,1.779364,1528.900000


In [169]:
Y_t.describe()

count    382.000000
mean       0.505236
std        0.500628
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Class, dtype: float64

In [170]:
all_metrics_validation(model, X_t,Y_t)

Accuracy score: 0.9293193717277487
ROC AUC score: 0.9795487567508292
Recall score: 0.9067357512953368
